1.(Easy)

Импортим библиотеки

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.model_selection import train_test_split

Обучим модель DecisionTreeRegressor(), берем модель машины 'audi':

In [2]:
df1 = pd.read_csv('archive (2)/audi.csv')
x_train, x_test, y_train, y_test = train_test_split(df1[['year', 'mileage', 'tax', 'mpg', 'engineSize']], df1['price'],  train_size = 0.8, random_state=42)
model = DecisionTreeRegressor()
model.fit(x_train, y_train)
prediction = model.predict(x_test)
r2_sc = r2_score(prediction, y_test)
print("r2_score:", r2_sc)

r2_score: 0.917517259387678


In [3]:
importances = model.feature_importances_
print(importances)

[0.24097458 0.0659256  0.03974242 0.46176101 0.19159638]


Из этого можно сделать вывод: для моделей audi самым важным признаком при формировании цены является 'mpg' (удельный расход топлива), а наименее важными - 'mileage'(пробег) и 'tax'(налог).

2.(Normal)

Объединим файлы в один DataFrame:

In [4]:
import os
path = 'archive (2)/'
models = os.listdir(path)
models.remove('unclean focus.csv')
models.remove('unclean cclass.csv')
csv_models = [file for file in models if file.endswith('.csv')]
headers = set()
for file in csv_models:
    df = pd.read_csv(os.path.join(path, file))
    if not headers:
        headers = set(df.columns.values)
    else:
        headers &= set(df.columns.values)
merged_df = pd.DataFrame()
for file in csv_models:
    df = pd.read_csv(os.path.join(path, file))
    common_columns = list(headers.intersection(df.columns))
    merged_df = pd.concat([merged_df, df[common_columns]])
merged_df

,price,year,model,transmission,mileage,engineSize,fuelType
0,8000,2016,Focus,Manual,38852,1.0,Petrol
1,13400,2019,Focus,Manual,11952,1.0,Petrol
2,14600,2019,Focus,Manual,22142,1.5,Petrol
3,9450,2016,Focus,Manual,14549,1.6,Diesel
4,9999,2015,Focus,Manual,7010,1.6,Diesel
...,...,...,...,...,...,...,...
4855,8680,2016,I30,Manual,25906,1.6,Diesel
4856,7830,2015,I40,Manual,59508,1.7,Diesel
4857,6830,2017,I10,Manual,13810,1.0,Petrol
4858,13994,2018,Tucson,Manual,23313,1.6,Petrol


Преобразуем категориальные признаки в числовые. Посмотрим, для начала, какие значения находятся в категориальных признаках:

In [5]:
unique_transmission = merged_df['transmission'].unique()
print('unique_transmission', unique_transmission)
unique_fuel_type = merged_df['fuelType'].unique()
print('unique_fuel_type', unique_fuel_type)
unique_model_names = merged_df['model'].unique()
cnt = merged_df['model'].nunique()
print('models count: ', cnt)
print('unique_model_names', unique_model_names)

unique_transmission ['Manual' 'Automatic' 'Semi-Auto' 'Other']
unique_fuel_type ['Petrol' 'Diesel' 'Hybrid' 'Electric' 'Other']
models count:  195
unique_model_names [' Focus' ' Fiesta' ' Puma' ' Kuga' ' EcoSport' ' C-MAX' ' Mondeo' ' Ka+'
 ' Tourneo Custom' ' S-MAX' ' B-MAX' ' Edge' ' Tourneo Connect'
 ' Grand C-MAX' ' KA' ' Galaxy' ' Mustang' ' Grand Tourneo Connect'
 ' Fusion' ' Ranger' ' Streetka' ' Escort' ' Transit Tourneo' ' A1' ' A6'
 ' A4' ' A3' ' Q3' ' Q5' ' A5' ' S4' ' Q2' ' A7' ' TT' ' Q7' ' RS6' ' RS3'
 ' A8' ' Q8' ' RS4' ' RS5' ' R8' ' SQ5' ' S8' ' SQ7' ' S3' ' S5' ' A2'
 ' RS7' ' C Class' ' 5 Series' ' 6 Series' ' 1 Series' ' 7 Series'
 ' 2 Series' ' 4 Series' ' X3' ' 3 Series' ' X5' ' X4' ' i3' ' X1' ' M4'
 ' X2' ' X6' ' 8 Series' ' Z4' ' X7' ' M5' ' i8' ' M2' ' M3' ' M6' ' Z3'
 ' SLK' ' S Class' ' SL CLASS' ' G Class' ' GLE Class' ' GLA Class'
 ' A Class' ' B Class' ' GLC Class' ' E Class' ' GL Class' ' CLS Class'
 ' CLC Class' ' CLA Class' ' V Class' ' M Class' ' CL C

Используем One-Hot Encoding для преобразования категориальных признаков:

In [6]:
df_encoded = pd.get_dummies(merged_df, columns=['model', 'transmission', 'fuelType'], prefix=['model', 'transmission', 'fuelType'], drop_first=True)


Подберем лучшие параметры для RandomForestRegressor():

In [7]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor()
param_grid = {
    'n_estimators': [100, 150],
    'max_depth': [5, 10, 15],
    'min_samples_split': [5, 10]
}
grid_search = GridSearchCV(estimator=forest, param_grid=param_grid, cv=5, error_score='raise')
df_weights = [col for col in df_encoded.columns if col != 'price']
X_train, X_test, Y_train, Y_test = train_test_split(df_encoded[df_weights], df_encoded['price'], train_size = 0.8, random_state=42)
grid_search.fit(X_train, Y_train)
print("Best parameters:", grid_search.best_params_, '\n', "Best score:", grid_search.best_score_)

Best parameters: {'max_depth': 15, 'min_samples_split': 5, 'n_estimators': 150} 
 Best score: 0.920690116727134


Вот наилучшие параметры для RandomForestRegressor(). Показатель на метрике r2 оказался при таких гиперпараметрах у случайного леса выше, чем у DecisionTreeRegressor().